In [84]:
import ndjson as json
from nltk.tokenize import word_tokenize
from nltk import tokenize
from collections import Counter

In [2]:
def custom_strip(s):
    for i in range(len(s)):
        if s[i].islower():
            start = i
            break
    for i in range(len(s))[::-1]:
        if s[i].islower():
            end = i
            break
    stripped = s[start:end+1]
    return stripped

In [3]:
def clean(token):
    token = token.lower()
    if token.isalpha() or token == "'s": return token
    if len(''.join(filter(str.isalnum, token))) == 0: return token
    if token.isdigit(): return "N"
    if any(c.isalpha() for c in token): return custom_strip(token)
    return "N"

In [90]:
all_tokens = []

with open('poe-sent.txt', 'r') as f:
    with open("poe.txt", 'w') as g:
        for line in f:
            lst = list(map(clean, word_tokenize(line)))
            g.write(" ".join(lst) + '\n')
            all_tokens.extend(lst)

In [97]:
def getvocab(counter, sz):
    mostcommon = [tup[0] for tup in counter.most_common(sz)]
    return mostcommon

def make_dataset(all_tokens, sz, l):
    counter = Counter(all_tokens)
    vocab = getvocab(counter, sz)
    with open('poe-sent.txt', 'r') as f:
        with open('poe-v%d-l%d.txt' % (sz, l), 'w') as g:
            lines = f.read().splitlines()
            tokens = lines[0].split(" ")
            merged = [token if token in vocab else "_UNK" for token in tokens]
            for i in range(1, len(lines)):
                tokens = lines[i].split(" ")
                tokens = [token if token in vocab else "_UNK" for token in tokens]
                if len(merged) + len(tokens) <= l:
                    merged.extend(tokens)
                else:
                    if merged.count("_UNK") / len(merged) < 0.15:
                        g.write(" ".join(merged) + '\n')
                    merged = tokens.copy()
            if len(merged) > 0 and merged.count("_UNK") / len(merged) < 0.15:
                g.write(" ".join(merged) + '\n')
#             for line in lines:
#                 tokens = line.split(" ")
#                 merged = [token if token in vocab else "_UNK" for token in tokens]
#                 if merged.count("_UNK") / len(merged) < 0.15:
#                     g.write(" ".join(merged) + '\n')

In [106]:
make_dataset(all_tokens, 18000, 100)